# Limpieza de datos de medicamentos

En este notebook se realiza la limpieza del conjunto de datos.
Se preparan los datos para el análisis eliminando variables no relevantes y verificando la estructura final del DataFrame.


In [10]:
import pandas as pd
import numpy as np

In [12]:
df = pd.read_csv('data_medicamentos.csv')
df.head()

,principio_activo,unidad_de_dispensacion,concentracion,unidad_base,nombre_comercial,fabricante,precio_por_tableta,factoresprecio
0,Midazolam,Ampolla,Midazolam 15 mg,ml,Dormicum,Siegfried,11199.800000,Alto
1,Acido Valproico,Tableta,Divalproato Sodico 500 mg,mg,Valcote,Lafrancol,3752.866667,Medio
2,Acido Valproico,Tableta,Divalproato Sodico 500 mg,mg,Valcote,Lafrancol,1777.266522,Medio
3,Fluoxetina,Capsula,Fluoxetina 20 mg,mg,Fluoxetina,Genfar,329.295281,Medio
4,Proximetacaina,Frasco,Proximetacaina 5 mg,ml,Alcaine,Alcon,64184.745760,Medio


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   principio_activo        10000 non-null  object 
 1   unidad_de_dispensacion  10000 non-null  object 
 2   concentracion           10000 non-null  object 
 3   unidad_base             10000 non-null  object 
 4   nombre_comercial        10000 non-null  object 
 5   fabricante              10000 non-null  object 
 6   precio_por_tableta      10000 non-null  float64
 7   factoresprecio          10000 non-null  object 
dtypes: float64(1), object(7)
memory usage: 625.1+ KB


In [15]:
#Manejo de valores nulos
df.isnull().sum()

,0
principio_activo,0
unidad_de_dispensacion,0
concentracion,0
unidad_base,0
nombre_comercial,0
fabricante,0
precio_por_tableta,0
factoresprecio,0


Se realizó una verificación de valores nulos en todas las columnas con isnull().sum().
Los resultados indicaron que no existen valores faltantes en ninguna de las variables, por lo cual no fue necesario aplicar técnicas de eliminación de datos nulos.

In [16]:
df['precio_por_tableta'].describe()

,precio_por_tableta
count,1.000000e+04
mean,1.626063e+05
std,2.860363e+06
min,1.025928e-01
25%,1.001897e+03
50%,3.695906e+03
75%,1.700168e+04
max,2.571032e+08


El conjunto de datos presenta una distribución altamente asimétrica, con valores extremos que influyen significativamente en la media y la desviación estándar, lo cual justifica la aplicación de técnicas de detección y eliminación de outliers antes del análisis estadístico y la visualización.

Al analizar la variable precio por tableta, se identificaron valores extremadamente altos, como un precio máximo superior a 2.5 × 10⁸ (257´103.200 pesos ó doscientos cincuenta y siete millones ciento tres mil doscientos pesos), los cuales no son coherentes con el contexto farmacéutico. Por esta razón, se aplicará el método del rango intercuartílico (IQR) para identificar y eliminar valores atípicos, garantizando un análisis más  realista de los precios de los medicamentos.

In [17]:
#Rango intercuartilico
q1 = df['precio_por_tableta'].quantile(0.25)
q3 = df['precio_por_tableta'].quantile(0.75)
iqr = q3 - q1
limite_inferior = q1 - 1.5 * iqr
limite_superior = q3 + 1.5 * iqr
limite_inferior, limite_superior

(np.float64(-22997.776404999997), np.float64(41001.35290699999))

El límite inferior no tiene sentido porque da un precio negativo. En cambio el superior si tiene sentido, pues considera atípica cualquier tableta de más de 41 mil pesos.

In [18]:
df = df[
    (df['precio_por_tableta'] >= limite_inferior) &
    (df['precio_por_tableta'] <= limite_superior)
]
df.shape

(8358, 8)

Se dejaron 8358 datos y 8 columnas

In [19]:
df['precio_por_tableta'].describe()

,precio_por_tableta
count,8358.000000
mean,5932.410895
std,8174.078459
min,0.102593
25%,790.000000
50%,2414.426041
75%,7442.632249
max,41000.000000


Luego de los cambios realizados, se obtienen resultados más realistas, no obstante, el precio mínimo de 0,10 pesos por tableta sigue siendo extremo, por lo que se aplicó un filtro farmacéutico adicional para eliminar registros con precios irrealmente bajos como los menores a 50 pesos por tableta, garantizando que los datos sean representativos del mercado real de medicamentos.

In [21]:
df = df[df['precio_por_tableta'] >= 50]
df.shape

(8189, 8)

In [22]:
df['precio_por_tableta'].describe()

,precio_por_tableta
count,8189.000000
mean,6054.368951
std,8213.339368
min,50.046875
25%,856.000000
50%,2516.733333
75%,7642.946879
max,41000.000000


In [23]:
df.to_csv('data_medicamentos_limpios.csv', index=False)